In [1]:
import polars as pl
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

# Pre-procesamiento

- Descargar los datos https://drive.google.com/drive/folders/12EbE0PEEnCjUVjhzMDNa1W7M9tR4ZzwW?usp=share_link
- Hacer merge de los datos
```bash
awk -F ';' 'FNR==1 && NR!=1{next} {print}' *.csv | sed '/^$/d' > ../merged.csv
```

Diccionario de datos facilitado por [https://github.com/luchobenitez/monda-guasu](https://github.com/luchobenitez/monda-guasu)

|Campo|Descripción|Tipo de Dato|
|-----|-----------|------------|
|serialtarjeta|Identificador de la tarjeta. Puede ser nominal o no, por lo que se puede llegar a la identidad del pasajero. Para nuestro análisis asumimos que es un pasajero|hash md5|
|idsam|Identificador del chip instalado en el equipo verificador. No necesariamente permanece con el mismo equipo verificador. Puede cambiar de bus, inclusive de línea o de EOT. Asumimos que identifica al bus|string|
|fechahoraevento|Timestamp del inicio del viaje|timestamp|
|producto|Tipo de producto donde:  - CR. Crédito o saldo negativo de la tarjeta - ES. Uso Especial (Estudiante o discapacitado) - MO. Monedero o uso normal|string|
|montoevento|Monto del evento descontado de la tarjeta. |entero|
|consecutivoevento|Identificador consecutivo de numero de transacción realizado con la tarjeta identificada con serialtarjeta|entero|
|identidad|Se refiere al propietario del validador donde: - 1. VMT - 2. MAS - 3. JAHA |entero|
|tipoevento|Tipo de evento donde: - 4. Viaje Normal - 8. Devolución - 10. Recarga de la tarjeta. - 14. Devolucion. Utilizaremos solo los del tipo 4|punto flotante|
|longitude|Longitud geográfica|punto flotante|
|Latitude|Latitude geográfica|punto flotante|
|idrutaestacion|Identificador de ruta asociado a una EOT. Ver tabla adicional|string|
|tipotransporte|Tipo de transporte donde - 0. Bus municipal interno - 1. Normal  - 3. Diferencial|entero|

# Analizando frequencias de eventos diarios

In [ ]:
df = pl.read_csv('merged.csv', separator=';', try_parse_dates=True, columns=['serialtarjeta', 'fechahoraevento'])
df.describe()

In [ ]:
df = df.with_column(pl.col("fechahoraevento").dt.weekday().alias("weekday"))
df = df.with_column(pl.col("fechahoraevento").dt.hour().alias("hour"))

grouped_df = df.groupby(["weekday", "hour"]).agg([pl.count("fechahoraevento").alias("dia_hora_evento_count")])
grouped_df

In [ ]:
# Assuming your grouped DataFrame is named `grouped_df`
# and has columns named `weekday`, `hour`, and `fechahoraevento_count`
grouped_df_pd = grouped_df.to_pandas()

# Map weekday numbers to names
weekday_map = {1: "Monday", 2: "Tuesday", 3: "Wednesday", 4: "Thursday", 5: "Friday", 6: "Saturday", 7: "Sunday"}
grouped_df_pd["weekday_name"] = grouped_df_pd["weekday"].map(weekday_map)

# Specify the order of the categories
weekday_order = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
hour_order = list(range(24))

In [ ]:
fig = px.bar(data_frame=grouped_df_pd, x="weekday_name", y="dia_hora_evento_count", color="hour", category_orders={"weekday_name": weekday_order, "hour": hour_order}, barmode='group')
fig.show()
# save to html
fig.write_html("dia_hora_evento_count.html")

In [ ]:
# Create a distribution plot of the serialtarjeta column
fig = px.histogram(df2, x='serialtarjeta', nbins=100, title='Distribution of serialtarjeta')

# Show the plot
fig.show()